In [33]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [34]:
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
features = newdf.to_numpy()
emissions = df['CO2E'].to_numpy()

In [35]:
def modelScore(X, y, model):
    avgR2 = avgSSE = avgMAPE = 0
    #cross validation using K folds
    kFolds = 5
    for i, (trainIndex, testIndex) in enumerate(KFold(n_splits=kFolds, shuffle=True, random_state=42).split(X)):
        X_train = X[trainIndex]
        X_test = X[testIndex]
        y_train = y[trainIndex]
        y_test = y[testIndex]
    

        if model == 'linear regression':
            fittedModel = LinearRegression().fit(X_train, y_train)
                
        elif model == 'decision tree':
            fittedModel = tree.DecisionTreeRegressor().fit(X_train, y_train)
                
        elif model == 'random forest':
            y_train = y_train.flatten()
            fittedModel = RandomForestRegressor().fit(X_train, y_train)

        elif model == 'support vector regression':
            y_train = y_train.flatten()
            fittedModel = SVR(kernel = 'linear').fit(X_train, y_train)    
    
        elif model == 'polynomial regression':
            polyX = PolynomialFeatures(degree=2).fit_transform(X)
            fittedModel = LinearRegression().fit(polyX[trainIndex], y_train)
            X_test = polyX[testIndex]

        elif model == 'xgboost':
            fittedModel = xgb.XGBRFRegressor(objective ='reg:linear', n_estimators = 10, seed = 123).fit(X_train, y_train)
        
        elif model =='lasso':
            fittedModel = Lasso().fit(X_train, y_train)
            

        y_pred = fittedModel.predict(X_test)
        
        #scores
        avgR2 += r2_score(y_test, y_pred)    
        avgSSE += np.sum((y_test - y_pred) ** 2)
        avgMAPE += mean_absolute_percentage_error(y_test, y_pred)

    
    avgR2, avgSSE, avgMAPE = avgR2 / kFolds, avgSSE / kFolds, avgMAPE / kFolds
   #print results
    print(model)
    print("R2 = " + str(avgR2))
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))

In [36]:
modelScore(features, emissions, 'linear regression')

linear regression
R2 = 0.9132785915590607
SSE = 172.6195696345183
MAPE = 0.04614202622279183


In [37]:
modelScore(features, emissions, 'decision tree')

decision tree
R2 = 0.7349076533982959
SSE = 564.4
MAPE = 0.08123144375218978


In [38]:
modelScore(features, emissions, 'random forest')

random forest
R2 = 0.8326354628823955
SSE = 416.40146000000004
MAPE = 0.06694322809314376


In [39]:
modelScore(features, emissions, 'support vector regression')

support vector regression
R2 = 0.8783182219644387
SSE = 231.72982845335702
MAPE = 0.0474385532925264


In [40]:
modelScore(features, emissions, 'polynomial regression')

polynomial regression
R2 = 0.5193065980378624
SSE = 1344.874736766754
MAPE = 0.07132996748624869


In [41]:
modelScore(features, emissions, 'xgboost')

xgboost
R2 = 0.772752120488174
SSE = 515.1256669329654
MAPE = 0.0773439495934521


c:\Users\emdua\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:03:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\emdua\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:03:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
c:\Users\emdua\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:160: UserWarning: [19:03:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objectiv

In [42]:
modelScore(features, emissions, 'lasso')

lasso
R2 = 0.8937950770555275
SSE = 218.70764501772538
MAPE = 0.04809102527020786


In [43]:
modelsList = ['linear regression', 'decision tree', 'random forest', 'support vector regression', 'polynomial regression']
multipleFeatures = df[[ 'STORAGE (gb)', 'number of camera (back)']].to_numpy()

featuresList = ['DISPLAY (inch)', 'STORAGE (gb)', 'MEMORY (gb)', 'CPU (cores)', 'GPU (cores)', 'NEURAL ENGINE (cores)', 'WEIGHT (gm)', 'FRONT CAMERA (mp)', 'BATTERY (mAh)', 'number of camera (back)', 'Sum of megapixels', 'MAIN CAMERA (mp)']

In [44]:
for model in modelsList:
    modelScore(multipleFeatures, emissions, model)

linear regression
R2 = 0.736895461662073
SSE = 585.791024593187
MAPE = 0.08779224499789329
decision tree
R2 = 0.6249690028484091
SSE = 826.301850340136
MAPE = 0.10498478316029494


random forest
R2 = 0.6281001870860641
SSE = 819.8997562144692
MAPE = 0.10482669944738914
support vector regression
R2 = 0.7100864306411325
SSE = 622.3806998507964
MAPE = 0.08137074548994702
polynomial regression
R2 = 0.7070934939840473
SSE = 650.9974096231292
MAPE = 0.09356856194059994


In [45]:
"""for x in featuresList:
    print(x)
    for model in modelsList:
        print(modelScore(df[x].to_numpy().reshape(-1, 1), emissions, model))
    print("")  """

'for x in featuresList:\n    print(x)\n    for model in modelsList:\n        print(modelScore(df[x].to_numpy().reshape(-1, 1), emissions, model))\n    print("")  '